API Key: SweUmsit7q4bITjAHGJbHRlWs
API Secret Key: dDpm4VZc48MMUgp5s8JrpdozM1YrKO3PBUKKMeEXTTQAJGCeUx
Bearer Token: AAAAAAAAAAAAAAAAAAAAAJmjLAEAAAAAzl9HA9NDExmrahuPmTmxF8XZSEQ%3D2hsAYb5058B25ue4B5XW8NErOtQAlzcp6HQzjTEF7a4t2hY9Mz


# TwitterAPI Setup

In [2]:
import os #collection of operating system interfaces
import tweepy as tw #library for TwitterAPI
import pandas as pd #data analysis package


In [3]:
#global to store each key
consumer_key= 'SweUmsit7q4bITjAHGJbHRlWs'
consumer_secret= 'dDpm4VZc48MMUgp5s8JrpdozM1YrKO3PBUKKMeEXTTQAJGCeUx'
access_token= '1273017906281021440-UswJnLXQblHPaFbcH6JqO7UU0f2amL'
access_token_secret= '3QBeKVF1cZnApZKfEF3TGA11KxFpi3MykCLMa9X7ZibrW'


In [5]:
auth = tw.OAuthHandler(consumer_key, consumer_secret) #login info for twitter
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True) #creates instance of TwitterAPI


# Build Dataset

In [37]:
userID = "MagicJohnson" #set Twtiter user ID
user = api.get_user(userID) #returns the user information
tweets = []

#we get every tweet from Magic Johnson's account
raw_tweets = tw.Cursor(api.user_timeline, screen_name=userID, tweet_mode = "extended").items()

#creates a list of tweets
for tweet in raw_tweets:
    tweets.append(tweet.full_text)
    

<class 'list'>


18
